This notebook can be used as a skeleton for containerizing existing Jupyter Notebooks and exposing them via a REST API.

There are two special cells in this notebook:

1. The first cell containing code, tagged with "parameters". Each variable defined there is considered as a variable for the service. These variables will be exposed as input (i.e., you can set their values) at the REST API. You can define as many variables as you wish there.

2. The last cell containing code, tagged with "results". Everything written to stdout in that cell will be returned by the service as
the response of the service call.

# The Service

In [3]:
''' This cell is tagged with the "parameters" tag. Each variable defined here is considered as a variable for the service.
These variables will be exposed as input (i.e., you can set their values) at the REST API.
 You can define as many variables as you wish here. '''

DWAVE_TOKEN = ""
seed = 123
node_count = 30
travel_start = 1

In [ ]:
# This is an example for a service that utilizes D-Wave to calculate the shortest path in a graph visiting all nodes and
# returning to the start location (travel_start) also known as the Travelling Salesman Problem.
#
# You can replace the code in this cell with your own code, and may add further cells below.
#
# NOTE: The last cell containing code must be the cell tagged with "results".

import dwave_networkx as dnx
import networkx as nx
import dimod
from dwave.system.samplers import DWaveSampler
from dwave.system import LeapHybridSampler, DWaveSampler
import matplotlib.pyplot as plt
import random

def draw_graph(G):
    default_axes = plt.axes(frameon=True)
    plt.figure(1, figsize=(100, 20), dpi=60)
    plt.rcParams['figure.figsize'] = [20, 10]
    pos=nx.spring_layout(G)
    nx.draw_networkx(G, node_color='r', pos=pos, node_size=300, alpha=0.8, ax=default_axes)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=pos, edge_labels=edge_labels)
    plt.axis('off')
    plt.show()

def solution_cost(solution):
    sum = 0
    for i in range(1, len(solution)):
        sum += G.edges[i, i-1]['weight']
    return sum


N = node_count
G = nx.complete_graph(N, nx.Graph())
random.seed(seed)
for (start, end) in G.edges:
    G.edges[start, end]['weight'] = round(random.random(),2)

my_sampler = LeapHybridSampler(token=DWAVE_TOKEN)
solution = dnx.traveling_salesperson(G, my_sampler, start=travel_start, label=f"TSP (start={travel_start}, nodes={N})")

In [ ]:
''' This Cell is tagged with the "results" tag. Everything written to stdout will be returned by the service as
the response. '''

print(solution)

## Building a Docker Container Locally

To build your service container locally, use the `docker build` command.
> **Note:** Ensure you have Docker installed on your machine.

Alternatively, you can build your service container using GitLab’s CI/CD pipeline.
See the next section for more details.

---

## Setting up CI Pipeline on GitLab (Optional)

### 1. Create or Use an Existing GitLab Project

Create a new GitLab project or use an existing one.

   ![GitLab Project Settings](img/CI-settings.png?1)

### 2. Enable GitLab Runner

Navigate to **Project Settings &rarr; CI/CD &rarr; Runners** and enable a GitLab shell runner for the project.

### 3. Create Access Token for Docker Registry

Go to **Project Settings &rarr; Access Tokens**, and create an access token with read/write permissions to the Docker registry.

### 4. Set Up CI/CD Variables

Under **Project Settings &rarr; CI/CD &rarr; Variables**, add the following variables:

- **`ACCESS_TOKEN:`**
  Access token with read/write permissions to the container registry.

- **`DOCKER_USERNAME:`**
  Username for pushing and pulling Docker images.

- **`DOCKER_REGISTRY:`**
  Address of the Docker registry where images will be published.

- **`DOCKER_IMAGE:`**
  Address of the Docker registry and the name of the Docker image (without tags).

- **`RUNNER_TAG:`**
  Tag of the GitLab runner that will execute the jobs.

   ![GitLab CI Variables](img/CI-variables.png)

### 5. Import Service Directory into Repository

Import the contents of your service directory into the repository.

### 6. Pipeline Execution

The pipeline will automatically execute, building and publishing the Docker image to the specified registry.

### 7. Triggering the Pipeline

The pipeline will run on any code changes but can also be triggered manually via **Project &rarr; Deploy/Pipelines**.

## Setting up Deployment (CD Pipeline) on GitLab (Optional)

### 1. Create or Use an Existing GitLab Project

Follow the [steps for setting up a basic CI/CD pipeline](#setting-up-ci-pipeline-on-gitlab-(optional)) (steps 2, 3, and 4).

### 2. Configure GitLab Project Variables

Navigate to **Project Settings &rarr; CI/CD &rarr; Variables**, and add the following variables:

- **`KUBERNETES_CONFIG:`**
  Contains the Kubernetes configuration file used by `kubectl`, which holds information about clusters, users, and contexts.

- **`KUBERNETES_NAMESPACE:`**
  Specifies the name of the Kubernetes namespace where resources will be deployed. Ensure the namespace exists before deployment.

- **`KUBERNETES_SERVICE_NODEPORT:`**
  Defines the NodePort value for exposing a service on each node's IP at a static port. This allows external access to the service through the specified port, typically between `30000-32767`, enabling communication from outside the cluster.

- **`KUBERNETES_MANIFEST_LOCATION:`**
  Specifies the location of the Kubernetes manifests (YAML files) to be deployed.

   ![Kubernetes Variables](img/CI-k8s-variables.png)

### 3. Continue with CI Pipeline Setup

Proceed with the [remaining CI pipeline setup steps](#setting-up-ci-pipeline-on-gitlab-(optional)) (steps 5, 6, and 7).

### 4. Automatic Deployment Execution

Once the container is published, the deployment phase will automatically execute.
